**<font color="blue" size="6em">연습 문제</font>** 

<div class="alert alert-block alert-warning">
<font color="blue" size="3em"><b>연습 문제1</b></font><br>
zip 컬렉션에서 '실제 예제1: 1000만 이상의 state만 가져오기' 를
aggregation 사용하지 않고, 직접 python 으로 구현하기

In [1]:
import pandas as pd
import numpy as np
import pymongo
from pprint import pprint

In [2]:
username = 'choigww'
password = ''
connection = pymongo.MongoClient('mongodb://%s:%s@ec2-52-79-60-207.ap-northeast-2.compute.amazonaws.com' % (username, password))
db = connection.test_db

### mongodb_zip datastructure

In [27]:
from pprint import pprint

result = db.zip.find(limit=1)
for record in result:
    pprint(record)

{'_id': '01001',
 'city': 'AGAWAM',
 'loc': [-72.622739, 42.070206],
 'pop': 15338,
 'state': 'MA'}


In [13]:
mongodb_zip = db.zip.find()

# create empty dataframe to store whole records
df_zip_records = pd.DataFrame(columns=['_id', 'city', 'loc', 'pop', 'state'])

for record in mongodb_zip:
    
    # create empty dataframe to store each record
    df_record = pd.DataFrame(columns=['_id', 'city', 'loc', 'pop', 'state'])
    df_record['_id'] = record['_id']
    df_record['city'] = record['city']
    df_record['loc'] = record['loc']
    df_record['pop'] = record['pop']
    df_record['state'] = record['state']
    
    # update df_zip_records
    df_zip_records = pd.concat([df_zip_records, df_record], axis=0)

In [23]:
df_zip_groupby_state_popsum = df_zip_records[['state','pop']].\
                                groupby('state').sum().reset_index()

print(df_zip_groupby_state_popsum[\
                    df_zip_groupby_state_popsum['pop'] >= 1000*100*100])

   state       pop
4     CA  59509780
9     FL  25373288
10    GA  12956432
14    IL  22855152
15    IN  11088272
19    MA  12032850
22    MI  18590594
24    MO  10221296
27    NC  13257274
31    NJ  15460376
34    NY  35980804
35    OH  21693034
38    PA  23763286
43    TX  33969202
45    VA  12362958


<div class="alert alert-block alert-warning">
<font color="blue" size="3em"><b>연습 문제2</b></font><br>
import한 restaurant 데이터를 가지고 각 borough 마다 레스토랑이 몇 개있는지를 출력하기

### mongodb_restaurant datastructure

In [26]:
mongod_restaurant = db.restaurant.find(limit=1)
for record in mongod_restaurant:
    pprint(record)

{'_id': ObjectId('5a0c1dcfdbab54966363f5c0'),
 'address': {'building': '1007',
             'coord': [-73.856077, 40.848447],
             'street': 'Morris Park Ave',
             'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
             'grade': 'A',
             'score': 2},
            {'date': datetime.datetime(2013, 9, 11, 0, 0),
             'grade': 'A',
             'score': 6},
            {'date': datetime.datetime(2013, 1, 24, 0, 0),
             'grade': 'A',
             'score': 10},
            {'date': datetime.datetime(2011, 11, 23, 0, 0),
             'grade': 'A',
             'score': 9},
            {'date': datetime.datetime(2011, 3, 10, 0, 0),
             'grade': 'B',
             'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}


In [55]:
mongodb_count_res_by_borough = db.restaurant.aggregate([
      {'$group' : 
                   {
                    '_id' : '$borough',
                   'count_restaurant' : {'$sum' : 1}
                   } 
      },
      {'$project':
                   {
                    '_id' : 0,
                    'borough' : '$_id',
                    'count_restaurant' : 1
                   }
          
      }
])

for record in mongodb_count_res_by_borough:
    print(record)

{'count_restaurant': 51, 'borough': 'Missing'}
{'count_restaurant': 969, 'borough': 'Staten Island'}
{'count_restaurant': 6086, 'borough': 'Brooklyn'}
{'count_restaurant': 2338, 'borough': 'Bronx'}
{'count_restaurant': 5656, 'borough': 'Queens'}
{'count_restaurant': 10259, 'borough': 'Manhattan'}


<div class="alert alert-block alert-warning">
<font color="blue" size="3em"><b>연습 문제3</b></font><br>
address의 zipcode별로 'Brazilian' 레스토랑이 몇개 있는지 계산하기

In [42]:
mongodb_count_brz_res_by_zipcode = db.restaurant.aggregate([
      
    {'$match' : {'cuisine' : 'Brazilian'}},
    {'$group' : 
                   {
                    '_id' : '$address.zipcode',
                   'count_brz_res' : {'$sum' : 1}
                   }
    },
    {'$project':
        { 
            '_id' : 0,
            'zipcode' : "$_id",
            'count_brz_res' : 1
        }
    }
])

for record in mongodb_count_brz_res_by_zipcode:
    print(record)

{'count_brz_res': 1, 'zipcode': '11238'}
{'count_brz_res': 3, 'zipcode': '11106'}
{'count_brz_res': 1, 'zipcode': '10002'}
{'count_brz_res': 2, 'zipcode': '10014'}
{'count_brz_res': 2, 'zipcode': '11101'}
{'count_brz_res': 1, 'zipcode': '11377'}
{'count_brz_res': 1, 'zipcode': '10034'}
{'count_brz_res': 1, 'zipcode': '11368'}
{'count_brz_res': 1, 'zipcode': '10009'}
{'count_brz_res': 4, 'zipcode': '10036'}
{'count_brz_res': 1, 'zipcode': '10128'}
{'count_brz_res': 3, 'zipcode': '10019'}
{'count_brz_res': 2, 'zipcode': '10065'}
{'count_brz_res': 2, 'zipcode': '11211'}
{'count_brz_res': 1, 'zipcode': '11103'}


<div class="alert alert-block alert-warning">
<font color="blue" size="3em"><b>연습 문제4</b></font><br>
평균평점이 가장 높은 레스토랑을 5개 출력하기

In [65]:
mongodb_top5_scored_restaurant = db.restaurant.aggregate([
    
    {'$unwind' : '$grades'},
    
    {'$group' : 
                   {
                    '_id' : '$name',
                   'avg_scores' : {'$avg' : '$grades.score'},
                   'cuisine_' : {'$addToSet' : '$cuisine'}
                   }
    },
    
    {'$unwind' : '$cuisine_'},
    
    {'$sort' : {'avg_scores' : -1}},
    {'$limit' : 5},
    {'$project' : {
                  '_id' : 0,
                  'name' : '$_id',
                  'avg_scores' : 1,
                  'cuisine_' : 1
    }}
])

for record in mongodb_top5_scored_restaurant:
    print(record)

{'avg_scores': 75.0, 'cuisine_': 'Juice, Smoothies, Fruit Salads', 'name': 'Juice It Health Bar'}
{'avg_scores': 73.0, 'cuisine_': 'Chinese', 'name': 'Golden Dragon Cuisine'}
{'avg_scores': 69.0, 'cuisine_': 'Juice, Smoothies, Fruit Salads', 'name': "Chelsea'S Juice Factory"}
{'avg_scores': 69.0, 'cuisine_': 'Bakery', 'name': 'Palombo Pastry Shop'}
{'avg_scores': 61.0, 'cuisine_': 'Middle Eastern', 'name': 'Koyla'}


<div class="alert alert-block alert-warning">
<font color="blue" size="3em"><b>연습 문제5</b></font><br>
cusine별로 평점이 가장 높은 레스토랑 출력하기

In [54]:
mongodb_top5_scored_restaurant_by_cuisine = db.restaurant.aggregate([
      
    {'$unwind' : '$grades'},
    
    {'$group' : 
                   {
                    '_id' : {'name' : '$name'},
                    'avg_score' : {'$avg' : '$grades.score'},
                    'cuisine_' : {'$addToSet' : '$cuisine'},
                    'name_' : {'$addToSet' : '$name'}
                   }
    
    },
    
    {'$unwind' : '$cuisine_'}, {'$unwind' : '$name_'},
    
    {'$sort' : {'avg_score' : -1}},
    
    {'$group' : 
                   {
                    '_id' : '$cuisine_',
                    'best_restaurant' : {'$first' : '$name_'},
                    'max_score' : {'$first' : '$avg_score'}
                   }
    },
    {'$project' : {
                  '_id' : 0,
                  'cuisine' : '$_id',
                  'best_restaurant' : 1,
                  'max_score' : 1
    }}
])

for record in mongodb_top5_scored_restaurant_by_cuisine:
    print(record)

{'best_restaurant': 'Corwn Fried Chicken And Fish', 'max_score': 6.0, 'cuisine': 'Chilean'}
{'best_restaurant': 'Fulton Hot Dog King', 'max_score': 11.666666666666666, 'cuisine': 'Hotdogs'}
{'best_restaurant': 'Laduree', 'max_score': 12.0, 'cuisine': 'Nuts/Confectionary'}
{'best_restaurant': "Mike'S Coffee Shop", 'max_score': 12.076923076923077, 'cuisine': 'CafÃ©/Coffee/Tea'}
{'best_restaurant': 'Bohemian Beer Garden', 'max_score': 12.166666666666666, 'cuisine': 'Czech'}
{'best_restaurant': 'Circle Line Xvii', 'max_score': 13.0, 'cuisine': 'Hotdogs/Pretzels'}
{'best_restaurant': 'Salad Pangea', 'max_score': 13.666666666666666, 'cuisine': 'Salads'}
{'best_restaurant': 'Bluestone Lane Coffee', 'max_score': 14.2, 'cuisine': 'Australian'}
{'best_restaurant': 'Zoe Cafe', 'max_score': 14.5, 'cuisine': 'Egyptian'}
{'best_restaurant': 'Bakhter Afghan Halal Kabab', 'max_score': 15.833333333333334, 'cuisine': 'Afghan'}
{'best_restaurant': 'Pao', 'max_score': 16.0, 'cuisine': 'Portuguese'}
{'best